In [ ]:
# --- Standardní knihovny ---
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Datasets ---
from sklearn.datasets import (
    make_blobs,
    make_circles,
    make_moons,
    load_digits,
    load_breast_cancer,
    fetch_california_housing
)

# --- Preprocessing a Pipeline ---
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA
from sklearn.pipeline import Pipeline

# --- Model Selection ---
from sklearn.model_selection import train_test_split, GridSearchCV

# --- Modely (SVM rodina) ---
from sklearn.svm import SVC, SVR, OneClassSVM

# --- Metriky ---
from sklearn.metrics import (
    # Klasifikace
    accuracy_score,
    classification_report,
    confusion_matrix,
    # Regrese
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    mean_absolute_percentage_error
)

In [ ]:
def plot_decision_boundary(model, X, y, title, plot_SV=True):
    x_min, x_max = X[:,0].min() - 0.5, X[:,0].max() + 0.5
    y_min, y_max = X[:,1].min() - 0.5, X[:,1].max() + 0.5

    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, 500),
        np.linspace(y_min, y_max, 500)
    )

    # Získáme "vzdálenost" od nadroviny pro mřížku
    # (Pipeline automaticky škáluje vstup)
    Z = model.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(7, 6))

    # 1. Vykreslení barevného pozadí (rozhodovací oblasti)
    plt.contourf(xx, yy, Z, alpha=0.2, cmap="coolwarm", levels=np.linspace(Z.min(), Z.max(), 20))

    # 2. Vykreslení čar: Hranice (0) a Marginy (-1, 1)
    # Toto je klíčové pro vizuální kontrolu
    plt.contour(xx, yy, Z, colors=['k', 'k', 'k'], linestyles=['--', '-', '--'],
                levels=[-1, 0, 1], linewidths=[1, 2, 1])

    # 3. Vykreslení běžných bodů
    plt.scatter(X[:,0], X[:,1], c=y, cmap="coolwarm", edgecolor='k', s=50, alpha=0.6)

    if plot_SV:
      # --- Analýza Support Vektorů ---
      svc = model.named_steps['model']
      sv_indices = svc.support_
      sv_alphas = np.abs(svc.dual_coef_[0]) # Alphas (Lagrangeovy multiplikátory)
      C_param = svc.C

      # Rozdělení SV na "Na hraně" a "Uvnitř/Porušující"
      # Používáme malou toleranci pro float porovnání
      tolerance = 1e-4
      on_margin = sv_alphas < (C_param - tolerance)
      off_margin = sv_alphas >= (C_param - tolerance) # Ty co rovnají C

      # Získání souřadnic SV
      sv_points = X[sv_indices]

      # A) Vykreslení bodů PŘESNĚ NA MARGINU (černé kruhy)
      if np.any(on_margin):
          plt.scatter(sv_points[on_margin, 0], sv_points[on_margin, 1],
                      s=200, linewidth=2, facecolors='none', edgecolors='black',
                      label='SV: Na Marginu (alpha < C)')

      # B) Vykreslení bodů UVNITŘ MARGINU (červené kruhy)
      if np.any(off_margin):
          plt.scatter(sv_points[off_margin, 0], sv_points[off_margin, 1],
                      s=200, linewidth=2, facecolors='none', edgecolors='red', linestyle=':',
                      label='SV: Uvnitř Marginu (alpha = C)')

    plt.title(title)
    plt.xlabel("x1")
    plt.ylabel("x2")
    if plot_SV:
      plt.legend(loc='upper right')
    plt.tight_layout()
    plt.show()

In [ ]:
X, y = make_blobs(n_samples=600, centers=2, cluster_std=0.8, random_state=7) # Zvýšil jsem trochu noise, aby byly nějaké body uvnitř

linear_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("model", SVC(kernel="linear", C=1.0))
])

linear_clf.fit(X, y)

plot_decision_boundary(linear_clf, X, y, "Rozlišení typů Support Vektorů")

In [ ]:
X, y = make_circles(n_samples=600, factor=0.3, noise=0.05, random_state=7)

# SVM s polynomiálním kernelem
poly_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("model", SVC(kernel="linear", C=1.0))
])
poly_clf.fit(X, y)

plot_decision_boundary(poly_clf, X, y, "Linear kernel, circles")

In [ ]:
X, y = make_circles(n_samples=600, factor=0.3, noise=0.05, random_state=7)

# SVM s polynomiálním kernelem
poly_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("model", SVC(kernel="poly", degree=2, coef0=1, C=1.0))
])
poly_clf.fit(X, y)

plot_decision_boundary(poly_clf, X, y, "Poly kernel, circles")

In [ ]:
X, y = make_circles(n_samples=600, factor=0.3, noise=0.05, random_state=7)

# SVM s RBF kernelem
poly_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("model", SVC(kernel="rbf", gamma='scale', coef0=5, C=1.0)) # scale = 1/(n_features*var(X))
])
poly_clf.fit(X, y)

plot_decision_boundary(poly_clf, X, y, "Poly kernel, circles")

In [ ]:
X, y = make_moons(n_samples=600,  noise=0.05, random_state=7)

poly_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("model", SVC(kernel="poly", degree=1, coef0=1, C=1.0))
])
poly_clf.fit(X, y)

plot_decision_boundary(poly_clf, X, y, "Poly kernel, moons")

In [ ]:
X, y = make_moons(n_samples=600,  noise=0.05, random_state=7)

poly_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("model", SVC(kernel="rbf", gamma='scale',  C=1.0)) # scale = 1/(n_features*var(X))
])
poly_clf.fit(X, y)

plot_decision_boundary(poly_clf, X, y, "Poly kernel, moons")

In [ ]:
X, y = make_moons(n_samples=800, noise=0.23, random_state=10)

X[:,0] = X[:,0] + 0.3*np.sin(3*X[:,1])

X[y==1,1] += 0.15

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=10, stratify=y
)

poly_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("model", SVC(kernel="poly", degree=3, C=1))
])
poly_clf.fit(X_train, y_train)

acc = accuracy_score(y_test, poly_clf.predict(X_test))
plot_decision_boundary(poly_clf, X_train, y_train, f"Polynomial kernel")
plot_decision_boundary(poly_clf, X_train, y_train, f"Polynomial kernel | Test data | acc={acc:.3f}", plot_SV=False)

In [ ]:
X, y = make_moons(n_samples=800, noise=0.23, random_state=10)

X[:,0] = X[:,0] + 0.3*np.sin(3*X[:,1])

X[y==1,1] += 0.15

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=10, stratify=y
)

poly_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("model", SVC(kernel="rbf", gamma=1, C=1))
])
poly_clf.fit(X_train, y_train)

acc = accuracy_score(y_test, poly_clf.predict(X_test))
plot_decision_boundary(poly_clf, X_train, y_train, f"RBF kernel")
plot_decision_boundary(poly_clf, X_train, y_train, f"RBF kernel | Test data | acc={acc:.3f}", plot_SV=False)

In [ ]:
X, y = load_breast_cancer(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=7, stratify=y
)

def evaluate(model):
    model.fit(X_train, y_train)
    acc = accuracy_score(y_test, model.predict(X_test))
    return acc

for gamma in [0.01, 0.1, 1, 3, 10]:
    clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm", SVC(kernel="rbf", C=1.0, gamma=gamma))
    ])
    print(f"gamma={gamma}: acc={evaluate(clf):.3f}")

In [ ]:
# 1. Načtení dat
X, y = load_breast_cancer(return_X_y=True)

# 2. Redukce dimenzí pomocí PCA na 2 komponenty (pro vizualizaci)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# 3. Split redukovaných dat
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.3, random_state=7, stratify=y
)

def plot_rbf_boundary(model, X, y, title, ax):
    """
    Vykreslí rozhodovací hranici, marginy a zvýrazní Support Vektory.
    """
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1

    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, 500),
        np.linspace(y_min, y_max, 500)
    )

    # Získání decision function pro vykreslení marginů
    Z = model.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Barevné pozadí
    ax.contourf(xx, yy, Z, alpha=0.2, cmap="coolwarm", levels=np.linspace(Z.min(), Z.max(), 20))

    # Vykreslení hranice (0) a marginů (-1, 1)
    ax.contour(xx, yy, Z, colors=['k', 'k', 'k'], linestyles=['--', '-', '--'],
                levels=[-1, 0, 1], linewidths=[0.8, 1.5, 0.8])

    # Vykreslení bodů (Trénovací data)
    scatter = ax.scatter(X[:, 0], X[:, 1], c=y, cmap="coolwarm", s=30, edgecolors='k', alpha=0.7)

    # --- Zvýraznění Support Vektorů ---
    # Musíme vytáhnout vnitřní model z Pipeline
    svc_model = model.named_steps['svm']
    sv_indices = svc_model.support_
    sv_points = X[sv_indices]

    # Vykreslení kroužků kolem SV
    ax.scatter(sv_points[:, 0], sv_points[:, 1], s=100,
               linewidth=1, facecolors='none', edgecolors='k', alpha=0.5,
               label='Support Vectors')

    ax.set_title(title)
    ax.set_xlabel("PC1")
    ax.set_ylabel("PC2")

# --- Hlavní smyčka pro různé hodnoty Gamma ---

# Nastavíme velikost grafu podle počtu modelů
gammas = [0.01, 0.1, 1, 10]
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

for i, gamma in enumerate(gammas):
    # Pipeline nyní obsahuje Scaler a SVM (PCA už proběhlo)
    clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm", SVC(kernel="rbf", C=1.0, gamma=gamma))
    ])

    # Trénování na PCA datech
    clf.fit(X_train, y_train)

    # Evaluace
    acc = accuracy_score(y_test, clf.predict(X_test))

    # Vykreslení
    title = f"Gamma={gamma}\nAcc={acc:.2f}"
    plot_rbf_boundary(clf, X_train, y_train, title, axes[i])

plt.tight_layout()
plt.show()

In [ ]:
# 1. Načtení a škálování dat
# U KernelPCA je kritické data nejdříve naškálovat, jinak kernel nefunguje správně!
X, y = load_breast_cancer(return_X_y=True)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2. Redukce dimenzí pomocí KernelPCA (RBF)
# gamma=0.04 je cca 1/n_features, dobrý start
kpca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04, random_state=42)
X_kpca = kpca.fit_transform(X_scaled)

# 3. Split redukovaných dat
X_train, X_test, y_train, y_test = train_test_split(
    X_kpca, y, test_size=0.3, random_state=7, stratify=y
)

def plot_boundary(model, X, y, title, ax):
    x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
    y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300), np.linspace(y_min, y_max, 300))

    Z = model.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    ax.contourf(xx, yy, Z, alpha=0.2, cmap="coolwarm", levels=np.linspace(Z.min(), Z.max(), 20))
    ax.contour(xx, yy, Z, colors=['k', 'k', 'k'], linestyles=['--', '-', '--'], levels=[-1, 0, 1], linewidths=[0.5, 1.5, 0.5])
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap="coolwarm", s=20, edgecolors='k', alpha=0.6)
    ax.set_title(title)
    # Osy u kPCA už nemají snadnou interpretaci, jsou to jen souřadnice v kernel prostoru
    ax.set_xticks(())
    ax.set_yticks(())

# --- Vykreslení pro různé gammy SVM modelu ---
# Gamma v kPCA je fixní (0.04), měníme jen gammu klasifikátoru

gammas_svm = [0.1, 1, 10, 50]
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

for i, g in enumerate(gammas_svm):
    # Tady už nepotřebujeme Scaler v pipeline, data jsou už naškálovaná a zredukovaná
    clf = SVC(kernel="rbf", C=1.0, gamma=g)
    clf.fit(X_train, y_train)

    plot_boundary(clf, X_train, y_train, f"SVM Gamma={g}\n(kPCA data)", axes[i])

plt.tight_layout()
plt.show()

In [ ]:
# 1. Načtení a příprava dat
digits = load_digits()
X, y = digits.data, digits.target

# Poznámka: Pro vizualizaci "znásilníme" data do 2D.
# Reálný model by měl mnohem vyšší přesnost na plných 64 dimenzích.
print(f"Původní tvar dat: {X.shape} (64 pixelů)")

# Pipeline pro přípravu dat (Škálování -> PCA na 2D)
preprocessor = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=2, random_state=42))
])

X_2d = preprocessor.fit_transform(X)

# Split na trénovací a testovací (už na 2D datech)
X_train, X_test, y_train, y_test = train_test_split(
    X_2d, y, test_size=0.3, random_state=42, stratify=y
)

# 2. Hledání nejlepšího SVM pomocí GridSearch
# U multiclass je RBF kernel skvělý pro vytváření "ostrůvků" pro jednotlivé číslice
param_grid = {
    "C": [1, 10, 100],
    "gamma": ['scale', 0.1, 1, 10]
}

print("Hledám nejlepší parametry pro 2D data...")
grid = GridSearchCV(SVC(kernel="rbf", decision_function_shape='ovo'), param_grid, cv=3)
grid.fit(X_train, y_train)

best_clf = grid.best_estimator_
acc = accuracy_score(y_test, best_clf.predict(X_test))

print(f"Nejlepší parametry: {grid.best_params_}")
print(f"Přesnost na testovacích datech (2D): {acc:.2%}")

# 3. Funkce pro vykreslení Multiclass hranic
def plot_multiclass_boundary(model, X, y, title):
    # Rozsah grafu s malou rezervou
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1

    # Jemná mřížka
    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, 500),
        np.linspace(y_min, y_max, 500)
    )

    # Predikce pro každý bod mřížky
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(10, 8))

    # A) Barevné regiony (používáme diskrétní colormapu 'tab10' pro 10 čísel)
    plt.contourf(xx, yy, Z, alpha=0.4, cmap='tab10')

    # B) Skutečné body
    scatter = plt.scatter(X[:, 0], X[:, 1], c=y, cmap='tab10', edgecolor='k', s=40, alpha=0.8)

    # Legenda (vytvoření legendy z barev scatter plotu)
    legend1 = plt.legend(*scatter.legend_elements(),
                        loc="best", title="Číslice")
    plt.gca().add_artist(legend1)

    plt.title(title)
    plt.xlabel("Principal Component 1")
    plt.ylabel("Principal Component 2")
    plt.grid(True, alpha=0.2)
    plt.tight_layout()
    plt.show()

# 4. Vykreslení
plot_multiclass_boundary(
    best_clf,
    X_train,
    y_train,
    f"SVM RBF Multiclass Decision Boundaries\n(Digits Dataset v 2D, Acc={acc:.2f})"
)

In [ ]:
# 1. Načtení dat
digits = load_digits()
X, y = digits.data, digits.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 2. Definice Pipeline
# Pipeline má 3 kroky: Škálování -> Redukce (kPCA) -> Klasifikace (SVM)
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("kpca", KernelPCA(n_components=2, kernel="rbf")), # Pevně 2 dimenze pro vizualizaci
    ("svm", SVC(kernel="rbf"))
])

# 3. Definice Gridu
# Musíme ladit gammu pro projekci (kpca) I gammu pro rozhodování (svm)
param_grid = {
    # Parametry pro KernelPCA
    "kpca__gamma": [0.01, 0.04, 0.1],  # Jak moc data "zakřivit" při projekci

    # Parametry pro SVM
    "svm__C": [1, 10, 100],            # Přísnost klasifikátoru
    "svm__gamma": ['scale', 0.1, 1]    # Dosah vlivu bodů v 2D prostoru
}

print("Spouštím Grid Search (kombinace projekce a klasifikace)...")
# n_jobs=-1 zapojí všechna jádra procesoru
grid = GridSearchCV(pipe, param_grid, cv=3, n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)

# 4. Vyhodnocení
best_model = grid.best_estimator_
acc = accuracy_score(y_test, best_model.predict(X_test))

print(f"\nNejlepší parametry: {grid.best_params_}")
print(f"Přesnost (Accuracy): {acc:.2%}")

# --- 5. Vizualizace ---
# Tady je to trochu trik: Pipeline umí predikovat rovnou z 64D dat.
# Ale abychom vykreslili 2D graf, musíme si vytáhnout mezivýsledek z kPCA.

def plot_grid_search_result(best_pipe, X_original, y_original, title):
    # A) Rozbalíme pipeline na jednotlivé kroky
    scaler = best_pipe.named_steps['scaler']
    kpca = best_pipe.named_steps['kpca']
    clf = best_pipe.named_steps['svm']

    # B) Transformujeme data do 2D (manuálně provedeme první dva kroky pipeline)
    X_scaled = scaler.transform(X_original)
    X_2d = kpca.transform(X_scaled)

    # C) Připravíme mřížku v tomto 2D prostoru
    x_min, x_max = X_2d[:, 0].min() - 0.1, X_2d[:, 0].max() + 0.1
    y_min, y_max = X_2d[:, 1].min() - 0.1, X_2d[:, 1].max() + 0.1

    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, 500),
        np.linspace(y_min, y_max, 500)
    )

    # D) Predikce: SVM už očekává 2D data (výstup z kPCA), takže mu dáme rovnou mřížku
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # E) Vykreslení
    plt.figure(figsize=(12, 8))
    plt.contourf(xx, yy, Z, alpha=0.3, cmap='tab10')
    scatter = plt.scatter(X_2d[:, 0], X_2d[:, 1], c=y_original, cmap='tab10', edgecolor='k', s=40, alpha=0.8)

    legend = plt.legend(*scatter.legend_elements(), loc="best", title="Číslice", ncol=2)
    plt.gca().add_artist(legend)

    # Do titulku vypíšeme vítězné parametry
    best_params_str = (f"kPCA Gamma: {best_pipe.get_params()['kpca__gamma']}\n"
                       f"SVM C: {best_pipe.get_params()['svm__C']}, "
                       f"SVM Gamma: {best_pipe.get_params()['svm__gamma']}")

    plt.title(f"{title}\n{best_params_str}")
    plt.axis('off') # Osy nemají fyzikální smysl
    plt.tight_layout()
    plt.show()

plot_grid_search_result(best_model, X_test, y_test, f"Best Model Visualization (Acc={acc:.2%})")

In [ ]:
# 1. Načtení plných dat
digits = load_digits()
X, y = digits.data, digits.target

print(f"Tvar dat: {X.shape} (64 dimenzí - žádná redukce)")

# 2. Rozdělení na trénovací a testovací sadu
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 3. Trénování (Pipeline + GridSearch)
# Na plných datech je SVM velmi efektivní.
# StandardScaler je stále nutný pro správnou funkci RBF kernelu.

pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC(kernel="rbf"))
])

param_grid = {
    "svm__C": [1, 10],             # Vyšší C obvykle pomáhá na čistých datech
    "svm__gamma": ['scale', 0.001] # 'scale' je default, 0.001 je klasika pro digits
}

print("Hledám nejlepší model na plných datech...")
grid = GridSearchCV(pipeline, param_grid, cv=3, n_jobs=-1)
grid.fit(X_train, y_train)

best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)

# 4. Vyhodnocení
acc = accuracy_score(y_test, y_pred)
print(f"\nNejlepší parametry: {grid.best_params_}")
print(f"Celková přesnost: {acc:.2%}")
print("-" * 60)

# Detailní report pro každou číslici
print("Classification Report:")
print(classification_report(y_test, y_pred))

# 5. Vizualizace chyb (Confusion Matrix)
# Protože nemůžeme kreslit hranice v 64D, podíváme se, kde model dělá chyby.
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title(f"Matice záměn (Confusion Matrix)\nAcc: {acc:.2%}")
plt.xlabel("Predikovaná číslice")
plt.ylabel("Skutečná číslice")
plt.show()

# 6. Zobrazení chyb (nepovinné, ale užitečné)
# Najdeme příklady, kde se model spletl
errors = (y_pred != y_test)
X_errors = X_test[errors]
y_pred_errors = y_pred[errors]
y_true_errors = y_test[errors]

if len(X_errors) > 0:
    print(f"\nUkázka chyb (Celkem {len(X_errors)} chyb z {len(y_test)} testovacích vzorků):")
    plt.figure(figsize=(10, 4))
    for i in range(min(5, len(X_errors))): # Ukážeme max 5 chyb
        plt.subplot(1, 5, i + 1)
        # Musíme data "odškálovat" nebo zobrazit původní surová data,
        # ale pro jednoduchost zobrazíme jen tvar, barvy mohou být posunuté scalerem
        # (zde bereme surová data z X_test, protože jsme scaler měli v pipeline)
        plt.imshow(X_errors[i].reshape(8, 8), cmap='gray_r')
        plt.title(f"T:{y_true_errors[i]}, P:{y_pred_errors[i]}", color='red')
        plt.axis('off')
    plt.show()
else:
    print("\nModel neudělal na testovacích datech žádnou chybu!")

In [ ]:
# --- 1. Příprava dat ---
data = fetch_california_housing()
X, y = data.data, data.target

# Pro rychlost v ukázce bereme jen vzorek (v praxi použijte vše)
X = X[:1000]
y = y[:1000]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=7
)

# --- 2. Trénování (Pipeline + GridSearch) ---
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("svr", SVR(kernel="rbf"))
])

param_grid = {
    "svr__C": [0.1, 1, 10, 100],
    "svr__gamma": ['scale', 0.1, 0.01],
    "svr__epsilon": [0.01, 0.1, 0.2]
}

print("Trénuji model...")
grid = GridSearchCV(pipe, param_grid, cv=3, n_jobs=-1)
grid.fit(X_train, y_train)

best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)

# --- 3. Výpočet Metrik ---
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print("\n" + "="*40)
print(f"Nejlepší parametry: {grid.best_params_}")
print("-" * 40)
print(f"RMSE (Root Mean Sq. Error): {rmse:.4f}")
print(f"MAE  (Mean Absolute Error): {mae:.4f}")
print(f"R²   (Score):               {r2:.4f}")
print(f"MAPE (Mean Abs. % Error):   {mape:.2%}")
print("="*40)

# --- 4. Vizualizace ---
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# A) Graf Skutečnost vs. Predikce
# Ideální stav: Všechny body leží na červené přerušované čáře
axes[0].scatter(y_test, y_pred, alpha=0.6, edgecolors='k', s=40)
min_val = min(y_test.min(), y_pred.min())
max_val = max(y_test.max(), y_pred.max())
axes[0].plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Ideální shoda')
axes[0].set_xlabel("Skutečné hodnoty (y_test)")
axes[0].set_ylabel("Predikované hodnoty (y_pred)")
axes[0].set_title(f"Actual vs Predicted (R²={r2:.2f})")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# B) Reziduální graf (Residual Plot)
# Co hledáme: Náhodný šum kolem nuly. Žádný "trychtýř" nebo "úsměv/mračení".
residuals = y_test - y_pred
axes[1].scatter(y_pred, residuals, alpha=0.6, edgecolors='k', s=40, c='purple')
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel("Predikované hodnoty")
axes[1].set_ylabel("Rezidua (Chyba)")
axes[1].set_title("Analýza Reziduí")
axes[1].grid(True, alpha=0.3)

# C) Histogram chyb
# Co hledáme: Zvonovitý tvar (Gaussova křivka) se středem v nule.
axes[2].hist(residuals, bins=20, edgecolor='k', alpha=0.7, color='skyblue')
axes[2].axvline(x=0, color='r', linestyle='--', lw=2)
axes[2].set_xlabel("Chyba (Reziduum)")
axes[2].set_ylabel("Počet výskytů")
axes[2].set_title("Rozdělení chyb")
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 1. Příprava dat
digits = load_digits()
X, y = digits.data, digits.target

# Tvoříme scénář "Novelty Detection":
# Trénovací data = Pouze číslice '0' (považujeme za "Normální")
X_train = X[y == 0]

# Testovací data = Mix '0' (Normální) a '4' (Anomálie)
# Bereme jen kousek od každého pro demonstraci
X_test_normal = X[y == 0][50:70]  # Další nuly, které model neviděl
X_test_anomaly = X[y == 4][:20]   # Čtyřky (pro model neznámé = anomálie)

# Spojíme testovací data dohromady
X_test = np.vstack([X_test_normal, X_test_anomaly])
# Vytvoříme labely jen pro barvení v grafu (model je nepotřebuje!)
y_test_labels = np.array([0]*20 + [4]*20)

# 2. Pipeline: Scaler -> PCA -> OneClassSVM
# Důležitý parametr 'nu':
# nu=0.05 znamená "Povoluji, aby cca 5% trénovacích dat leželo mimo bublinu" (tolerance šumu)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=2)),
    ("model", OneClassSVM(kernel="rbf", gamma=0.5, nu=0.05))
])

# Fitujeme POUZE na nulách (X_train)
pipeline.fit(X_train)

# 3. Predikce
# OneClassSVM vrací: +1 (Normální / Inlier), -1 (Anomálie / Outlier)
y_pred = pipeline.predict(X_test)

# Přepočet na čitelné výsledky
n_errors = (y_pred[y_test_labels == 0] == -1).sum() + (y_pred[y_test_labels == 4] == 1).sum()
print(f"Počet chyb na testovacích datech: {n_errors} z {len(X_test)}")

# 4. Vizualizace
def plot_anomaly_detection(model, X_train, X_test, y_test, title):
    preprocessor = Pipeline(model.steps[:-1]) # Vše kromě posledního kroku (SVM)
    clf = model.named_steps['model']

    X_train_2d = preprocessor.transform(X_train)
    X_test_2d = preprocessor.transform(X_test)

    # Mřížka
    x_min, x_max = min((X_train_2d[:, 0].min(), X_test_2d[:, 0].min())) - 1, max((X_train_2d[:, 0].max(), X_test_2d[:, 0].max())) + 1
    y_min, y_max = min((X_train_2d[:, 0].min(), X_test_2d[:, 0].min())) - 1 , max((X_train_2d[:, 0].max(), X_test_2d[:, 0].max())) + 5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 500), np.linspace(y_min, y_max, 500))

    # Predikce na mřížce
    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(10, 7))

    # A) Pozadí - Rozhodovací funkce
    # Červená oblast = Anomálie, Modrá oblast = Normální
    plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), 0, 7), cmap="Reds_r", alpha=0.3) # Outlier zóna
    plt.contourf(xx, yy, Z, levels=np.linspace(0, Z.max(), 7), cmap="Blues", alpha=0.3)  # Inlier zóna
    plt.contour(xx, yy, Z, levels=[0], linewidths=3, colors='black') # Hranice bubliny

    # B) Body - Trénovací data (Nuly) - malé bílé tečky
    plt.scatter(X_train_2d[:, 0], X_train_2d[:, 1], c='white', s=20, edgecolors='k', label="Trénovací data ('0')")

    # C) Body - Testovací data (Nuly vs Čtyřky)
    # Zelené = Normální testovací data ('0')
    # Červené = Anomálie ('4')
    norm_mask = (y_test == 0)
    anom_mask = (y_test == 4)

    plt.scatter(X_test_2d[norm_mask, 0], X_test_2d[norm_mask, 1], c='green', s=100, edgecolors='k', marker='o', label="Test: Normální ('0')")
    plt.scatter(X_test_2d[anom_mask, 0], X_test_2d[anom_mask, 1], c='red', s=100, edgecolors='k', marker='x', label="Test: Anomálie ('4')")

    plt.title(title)
    plt.legend(loc="upper right")
    plt.grid(True, alpha=0.3)
    plt.show()

plot_anomaly_detection(pipeline, X_train, X_test, y_test_labels, "One-Class SVM: Detekce nul (vše ostatní je anomálie)")